# Metalanguage evaluation

## Simplification

Simplification is a process by which a formula is converted to a simplest equivalent form; the lambda notebook has extensive, but highly imperfect and heuristic, support for automatic simplification. To quote the [SymPy docs](https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html): *“simplest” is not a well-defined term*.

### What is simplification?

To illustrate, consider double negation:

In [1]:
%te ~~p_t

¬¬p_t

If double negation is classical (which it is by default in the metalanguage), then it is relatively inarguable that this can be simplified: `p_t` is a structurally less complex formula that is easier to read by a human than `~~p`, and the "heuristic" corresponds to a standard logical inference rule (double negation elimination; $\neg \neg p \vdash p$). And indeed, the metalanguage can simplify this:

In [2]:
%te simplify ~~p_t

p_t

(This document heavily uses `_llast` -- as a reminder, when a `%te` magic is used, it exports its value to the notebook scope as the python `_llast`. This is especially handy for examining derivations.)

In [3]:
_llast.derivation

1. ~~p_t    (negation elimination)
 2. p_t

Here's a second obvious case, where a redundant conjunct can be eliminated:

In [4]:
%te simplify p_t & (q_t & p_t)

(p_t ∧ q_t)

Cases like this will automatically simplify as well, though by a slightly complex process, First, sequences of conjuncts are reordered so that identical conjuncts are adjacent ("alphabetic normalization"), then conjunction idempotence is checked, and then a binary conjunction expression is reconstructed ("join"):

In [5]:
_llast.derivation

1. (p_t & (q_t & p_t))    (alphabetic normalization)
 2. [p_t, p_t, q_t]    ([conjunction (idempotence)])
 3. [p_t, q_t]    (join on ∧)
 4. (p_t & q_t)

However, now consider the following restricted quantifier expression. As you can see, it does not automatically simplify:

In [6]:
%te simplify Forall x_e << {_c1, _c2, _c3} : P_<e,t>(x)

(Forall x_e << {_c3, _c2, _c1}: P_<e,t>(x_e))

However, this expression plainly *can* be converted to a non-quantified expression; it is equivalent to a conjunction. This can be forced:

In [7]:
_llast.simplify_all(eliminate_quantifiers=True)

((P_<e,t>(_c1) ∧ P_<e,t>(_c2)) ∧ P_<e,t>(_c3))

Is the result simpler? The answer is much more equivocal than the double negation case. Technically, it is a few characters shorter in latex form (though at the time of writing, the two have identical length `repr`s, 45 characters), and some might find it easier to read (it is a more standard classical predicate logic expression). It's not structurally simpler (the quantified expression has a flatter tree). In some contexts, especially if the restriction is derived, it may not capture the original intent of the formula. Moreover, as the restrictor size grows, the complexity relation between the two formulas changes. For all these reasons, the metalanguage simplification system will not aggressively eliminate quantifiers. Rather, it takes a **heuristic** approach: if the domain size is known to be <=1, it will apply qauntifier elimination, but otherwise, this must be manually triggered.

In general, the real world of automatic simplification is more like this latter case than the former. It is necessarily heuristic, while allowing a human to intervene in certain ways. 

### Outcomes of simplification

Despite all of the above, there are certain things that are guaranteed by simplification. Here are a few basic facts:

* Simplification will return `self` if there is no change (called a "noop" at various points in this document), otherwise it will return a modified copy.
* Except for certain special cases, simplification failures act like noops. In special cases (e.g. an `Iota` expression not shielded by a `Partial` condition), `lamb.meta.meta.DomainError` may be raised.
* Simplification will set or update a value for the member variable `.derivation`

There are also certain cases under which simplification is guaranteed to accomplish something. Let us call an expression **concrete** if it can be determined that its atoms consist only of `MetaTerm` elements. In other words: it has no *free terms* (regular constants and variables whose value is not determined to a `MetaTerm` value by an assignment). Let us call an expression **finite-safe** if it can be determined that all quantification-like expressions involve a finite, concrete domain. The metalanguage provides the following guarantee:

* All concrete and finite expressions simplify to a single `MetaTerm`.

This is perhaps easiest to see for boolean expressions. The following expression does not simplify by default without values for `p` and `q`:

In [8]:
%te simplify ~(p_t & ~q_t)

¬(p_t ∧ ¬q_t)

However, a similar expression with truth-values instead of variables will simplify (using classical propositional logical inferences):

In [9]:
%te simplify ~(True & ~True)

True

In [10]:
_llast.derivation.trace()

DisplayNode instance: HTML rendering only

This guarantee, as noted above, includes quantified expressions as long as the domain can be determined to be finite. (For more on setting domain restrictions, see the documentation on *Domain elements and MetaTerms*, as well as the section on Models below.) To illustrate this, we can simply use the domain for `t`:

In [11]:
%te simplify Exists p_t : ~p

True

Simplification here simply iterates over the domain `{True, False}` and checks the formula for every value of `p`, straightforwardly finding a verifier:

In [12]:
_llast.derivation.trace()

DisplayNode instance: HTML rendering only

Here's another example; this illustrates another point about quantification as well as a very general point about simplification. Simplification doesn't always find the path you might expect.

In [13]:
%te simplify Forall p_t : p | ~p

True

 From the above example, you may be in a mindset of iteration over the domain. This formula could be simplified this way, but it needn't be. In this case, the subformula `p | ~p` can be simplified on its own terms via the law of the excluded middle, leading to quantified expression with no bound variables in the body. Such quantified expressions are also guaranteed to simplify (modulo certain cases where the domain size matters and can't be determined from the formula):

In [14]:
_llast.derivation

1. (Forall p_t: (p_t | ~p_t))    ([excluded middle])
 2. (Forall p_t: True)    (trivial ∀ elimination)
 3. True

### Invoking simplification

There are several ways of invoking simplification, as well as many cases where it is invoked automatically.

1. In most `lamb.lang` contexts (e.g. `%%lamb` magics, composition) it is invoked automatically. This includes reduction.
2. In `%te` magics, it is not automatic, but can be triggered via the `simplify` option as illustrated above.
3. It can be triggered on `TypedExpr` objects directly via the member function `simplify_all` function as a primary interface, with a number of related functions discussed below.
4. It can be triggered on `TypedExpr` objects via the function `meta.simplify_all` (note: this is really an alias to `meta.ply.simplify_all`)

In general, `%te simplify stuff` is equivalent to `te("stuff").simplify_all()` or the equivalent absolute function call:

In [15]:
te("~~p_t").simplify_all()

p_t

In [16]:
meta.simplify_all(te("~~p_t"))

p_t

The function call versions support a number of options via named parameters. For example, `.simplify_all(reduce=True)` recursively applies beta reduction; equivalent to first calling `reduce_all()` as below.

* `reduce`: do all beta reductions (default: `False`)
* `evaluate`: if possible, evaluate certain expressions (mainly quantifiers) when they can be finitely checked against `MetaTerms`. (default: `True`)
* `collect`: for associative operators, do simplification strategies that can be applied to an entire sequence, rather than a pairwise sequence. (See `meta.ply.collect`, discussed below.) (default: `True`)
* `alphanorm`: do "alphabetic normalization", i.e. reorder terms so that they are in a stable order. Disabled by `collect=False`. (Default: `True`)
* `eliminate_sets`: try to eliminate set expressions where possible, typically be replacing them with quantified expressions. This uses set class `eliminate()` implementations (see discussion of `eliminate()` below). Eliminating sets is not always possible, and can make the formula unreadable by a human when it is possible. (Default: `False`)
* `eliminate_sets_all`: apply regular `eliminate_sets` strategies, with the addition of eliminating arbitrary finite `<<` expressions. These can always be eliminated by conversion to disjunctions  of equalities, but past cardinality ` are almost always guaranteed to make the formula more complex and less readable. (Default: `False`)
* `strict_charfuns`: when evaluating `MetaTerm` functions, whether to take mapping-based functions to be strict or not. If such a function is strict, it will raise `DomainError` when supplied a value not in its domain, otherwise it will return `False`. See discussion under *Models* below. (default: `True`)

All `TypedExpr` objects support the following related member functions, which may be noops:

**Reduction**

* `reduce_all`: recursively do all beta reductions in the expression (also triggerable via `reduce=True`).
* `reduce()` will do a single reduction step, and is a noop on `TypedExpr` expressions that are not directly reducible.
* `subreduce()`: given a *path* (consisting of a tuple of `int`s that provide indices into the expression's AST), do reduction of a subformula.

**Elimination**

* `eliminate()`: for classes where this is defined, eliminate the main operator (somehow) if present. This is heterogenous. For example, calling `.eliminate()` on a universally quantified expression, if the domain is finite, will convert it to a conjunction. This is often a noop.

As another example, `ExistsExact` has an `eliminate` implementation that converts it to a standard quantified expression using regular `Exists` and `Forall`. This is of course further eliminable in many cases.

In [17]:
te("ExistsExact p_t: p_t").eliminate()

(Exists p_t: (p_t & (Forall p1_t: (p1_t >> (p_t <=> p1_t)))))

In [18]:
te("ExistsExact p_t: p_t").simplify_all(eliminate_quantifiers=True).derivation # the full .trace() is a bit unwieldy

1. (ExistsExact p_t: p_t)    (∃! elimination)
 2. (Exists p_t: (p_t & (Forall p1_t: (p1_t >> (p_t <=> p1_t)))))    (∃p => ∨)
 3. ((False & (Forall p1_t: (p1_t >> (False <=> p1_t)))) | (True & (Forall p1_t: (p1_t >> (True <=> p1_t)))))    (Recursive simplification of operand 0)
 4. (False | (True & (Forall p1_t: (p1_t >> (True <=> p1_t)))))    (Recursive simplification of operand 1)
 5. (False | True)    (disjunction (elimination on `True`))
 6. True

**Partiality**

* `calculate_partiality()`: If there are `Partial` subexpressions, do inference to project them to the expression this is called on. For example, in the following case a `False` condition on a subexpression is projected to the main expression:

In [19]:
te("p_t & Partial(q_t, False)").calculate_partiality()

Partial((p_t & q_t), False)

In [20]:
te("p_t & Partial(q_t, False)").simplify_all()

(Partial(q_t, False) ∧ p_t)

## Execution and compilation

Simplification is generally not aimed at efficiency, but rather human understanding, and can be relatively slow. This is because simplification involves (a) step-wise transformation of formulas, and (b) tracking of derivation history at every point. For simplification sequences that involve many steps, these two factors can add substantial algorithmic overhead.

In cases where the goal is fast evaluation of a **concrete** formula (i.e. one with all free terms valued), the compilation/execution system provides an alternative. Essentially, given a parsed (and therefore type-safe) metalanguage expression, this system allows the expression to be "compiled" into reasonably fast python code that no longer has any of the overhead associated with metalanguage simplification or the type system. (It will still have greater overhead than writing comparable python code directly.)

### What is execution/compilation?

Consider the following simple metalanguage function, which is already concrete, so is guaranteed to simplify.

In [21]:
f = %te L x_n : x * 2
%time f(20).simplify_all(reduce=True)
# use %timeit for a more accurate (but more time-consuming to gather) estimate.
# >>>%timeit f(20).simplify_all(reduce=True)
# 125 µs ± 668 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

CPU times: user 1.03 ms, sys: 21 µs, total: 1.05 ms
Wall time: 1.06 ms


40

Simplification here is already fast enough to not be perceptible to a human (so this is a toy example for compilation). However, it can be observed that the raw compiled form, accessible via `lamb.meta.meta.compiled`, tends to be approximately 2 orders of magnitude faster.

In [22]:
compiled_f = meta.compiled(f, with_context={})
%time compiled_f(10)
# >>>%timeit compiled_f(10)
# 1.07 µs ± 2.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

CPU times: user 26 µs, sys: 1 µs, total: 27 µs
Wall time: 29.8 µs


20

Of course, for this toy example raw python code is easy to write and is yet again several orders of magnitude faster (`%timeit` for a simple equivalent closure reports `61.8 ns` on the same device that generated the above comments.) So the point of this is not generally to replace python code. Rather, the use of compilation is when writing type-safe/statically typed logic-oriented code that it may be useful to run many times.

As an example, here is an implementation of a version of Kratzer's ordering relation on possible worlds. Given some set of propositions and two worlds, it calculates which world is closer to the "ideal" determined by maximal consistent sets of those premises. This relation is very easy to write in the metalanguage in a way that directly corresponds to what is written in papers in the literature. It would be quite a bit less easy to implement this ordering relation as regular python code in a way that is type-safe, convertable to a formula, integrates with other metalanguage code, etc.

In [23]:
type_s = types.BasicType("s")
# finite restriction to 8 worlds
lang.get_system().add_basic_type(type_s)
type_s.domain = type_s.get_subdomain(count=2**3)
order = %te L prem_{{s}} : L w1_s : L w2_s : (Set p_{s} : p << prem & w1 << p) < (Set p_{s} : p << prem & w2 << p)
order

(λ prem_{{s}}: (λ w1_s: (λ w2_s: ((Set p_{s}: ((p_{s} << prem_{{s}}) & (w1_s << p_{s}))) < (Set p_{s}: ((p_{s} << prem_{{s}}) & (w2_s << p_{s})))))))

Unfortunately, simplification of this function with arguments is relatively slow; one call on a relatively fast computer at the time of writing is in the 200-300ms range, above the threshold of human perceptability. Even worse, this function needs to be checked against many values to be used as it is in the literature (as the "core" of an ordering semantics for modals).

In [24]:
# pick some arbitrary arguments
p1 = %te {_w0, _w1, _w2, _w3}
p2 = %te {_w0, _w1, _w4, _w5}
w1 = %te _w1
w2 = %te _w2
# this can be made even worse via `eliminate_sets_all=True`, adding about another order of magnitude
%time order({p1, p2})(w2)(w1).simplify_all(reduce=True, eliminate_sets=True)
# >>>%timeit order({p1, p2})(w2)(w1).simplify_all(reduce=True, eliminate_sets=True)
# 257 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

CPU times: user 272 ms, sys: 3.32 ms, total: 275 ms
Wall time: 274 ms


True

Of course, there may be ways to improve simplification (and contributions to the project along these lines are very welcome). But rather than worrying about this, we can simply compile this function and get much, much better performance; compiled metalanguage code provides something like an in-principle bound for how fast simplification could be, given current implementations of the operators in question.

In [25]:
compiled_order = meta.compiled(order({p1, p2}), with_context={})
compiled_w1 = meta.compiled(w1, with_context={})
compiled_w2 = meta.compiled(w2, with_context={})
%time compiled_order(compiled_w2)(compiled_w1)
# >>>%timeit compiled_order(w2)(w1)
# 769 µs ± 3.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

CPU times: user 1.36 ms, sys: 293 µs, total: 1.65 ms
Wall time: 1.37 ms


True

### Invoking compilation and executing compiled metalanguage

**Basic compilation**. The direct interface to compilation is the function **`lamb.meta.compiled`** (an alias to **`lamb.meta.meta.compiled`**), as seen in the above examples. This function returns a compiled version of a `TypedExpr` that it is passed; this compiled version is also cached directly on a `TypedExpr` object so that further calls to `compiled` on the same object can just return the cached version. You can override this by supplying `recompiled=True` to this function.

Compilation in general takes a `TypedExpr` and produces a function that can be supplied with a *context*. A context is simply a mapping that provides values for any free terms in the expression; executing the compiled expression without valueing all free terms wil result in a `KeyError` being raised. This is very similar to e.g. python `locals()`. No types are checked at the point of supplying a context, and access to the context is lazy. As you can see above, the wrapper function `meta.compiled` can take a pre-provided context via the `with_context` named parameter, which is especially useful for expressions with no free terms. If you set `validate=True`, this function will additionally check that the context supplies all free terms for the expression (with this option set, supplying no context is equivalent to supplying the empty context).

If the `TypedExpr` is functional, compilation will produce an object that, after the context is supplied, is a callable. Again, no types will be checked on application. The safest approach is to supply callables of this type with compiled arguments.

Advanced: Given some typed expression `e`, `e._compiled` accesses the compiled version of `e`, generating it if necessary.

**Compilation wrappers**. Using the above technique is the most speed-conscious way of dealing with compiled `TypedExpr` elements, but it is somewhat error-prone. For this reason there is wrapper code that imposes greater overhead in combination with more sanity checking. The overhead is still less than simplification, especially in involved cases like the ordering semantics example above. The primary interface to this, especially when aiming to produce callables, is **`lamb.meta.exec`** (an alias to **`lamb.meta.meta.exec`**). Given some `TypedExpr` and context, this will validate the context, and wrap the compiled version of the `TypedExpr` in a minimal dynamic type-checker for its function parameters. There are a few other bells and whistles that this document won't go into in detail, such as uncurrying and control of partial application. This function also allows some arguments to be pre-supplied, which will invoke full static type-checking. The context is supplied via named parameters. Here's an example:

In [26]:
# requires previous cells
compiled_order = meta.exec(order, te({p1, p2}))
compiled_w1 = meta.exec(w1)
compiled_w2 = meta.exec(w2)
compiled_order(w2)(w1)
# >>>%timeit compiled_order(w2)(w1)
# 777 µs ± 7.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

True

While the wrapper code generally adds overhead, the overhead is dependent only on the number of arguments, not the complexity of the formula involved, and so it doesn't particularly matter in the case of this example. To disable dynamic type-checking, provide the named parameter `_check_domain=False` to `exec`.

One final note: a compiled atomic `MetaTerm` corresponds directly to the underlying domain element (see documentation on MetaTerms and domain elements). So it is entirely possible to provide such arguments directly, rather than go through a compilation stage. For example, `meta.exec(te("_w1"))` is simply the python string `_w1`.

## Logical evaluation and assignments

By "evaluation" in this section, I mean the interpretation of some logical formulas against meta-values for those formulas. We have already seen one way in which evaluation is supported: if a formula consists only of `MetaTerm`s, including via an assignment, then either simplification or execution of compiled code can be used for complete evaluation. This section discusses various pieces of support cord that assist with this.

### Assignments

Assignments in the broad sense have been used throughout this document so they aren't a new concept; in general the word "assignment" in the lambda notebook refers to a mapping of *term names* to `TypedExpr` objects. (This is a bit different from how it is used in the linguistics literature, where it often refers to a function of type `<n,e>`.) Both variables and constants are supported in the domain of an assignment; `MetaTerm` names are not. Any such mapping object (e.g. a `dict`) is supported in most places where the metalanguage can make use of an assignment. All `TypedExpr` objects support the method **`under_assignment`**:

In [27]:
te("p_t & q_t").under_assignment({'p': True, 'q': True}).simplify_all()

True

Of course, assignments can be used for many things beyond evaluation, and there is no constraint to `MetaTerm`s in the range of the map. For example, binding expressions use assignments to do type inference over their bodies. The `under_assignment` function fully support type inference and type checking.

In [28]:
te("f_<X,t>(p_X)").under_assignment({'p': te("q_t")}).derivation

1. f_<X,t>(q_t)    (Type inference)
 2. f_<X,t>(p_X)    (Assignment substitution)
 3. f_<t,t>(q_t)

In [29]:
te("f_<X,t>(p_t)").under_assignment({'p': te("q_X")}).derivation

1. f_<t,t>(q_?10)    (Type inference)
 2. f_<t,t>(p_t)    (Assignment substitution)
 3. f_<t,t>(q_t)

In [30]:
with lamb.errors():
    te("f_<e,t>(p_e)").under_assignment({'p': te("q_t")})

<span style="color:red">**TypeMismatch**</span>: `p_e` and `q_t` conflict (Variable replace failed with mismatched types)

Values in the assignment may be polymorphic, and if they are `under_assignment` supports what is often termed "let polymorphism": distinct substitutions are allowed to resolve to distinct specialized types.

The class **`lamb.meta.Assignment`** (an alias for `lamb.meta.meta.Assignment`) provides some extra useful features on top of a standard mapping class (with implementation building on a [ChainMap](https://docs.python.org/3/library/collections.html#collections.ChainMap)). Assignment` objects implement the standard `collections.abc.Mapping` interface, with the following lambda notebook-specific tweaks:

* They enforce that values are `TypedExpr`s (the `te` function is applied to non-te values)
* They support lambda notebook rich reprs. (**Side note**: this style of rendering for mappings is generally available via `utils.dict_latex_repr`.) Following Heim and Kratzer, when in linearized form mappings are displayed as a sequence of `value/term` pairs.
* They support incremental update via several functions:
    - the standard mapping `update` function changes the map in place (no return).
    - the `modify` function changes the map in place while tracking change history, so that the prior mapping can be rendered and recovered. (No return.)
    - the `new_child` function (similar to the `ChainMap` api) returns a modified version of the current assignment, and does not change `self`.
    - the `merge` function overwrites term mappings while requiring the expressions to be compatible (it raises `TypeMismatch` if this fails). See below for details.
    - For modified assignments (via any of the three prior functions), the function `pop_child()` returns to the previous value. `pop_child` on an `Assignment` with only a single mapping will clear the map!
* The `flatten` function returns a new `Assignment` that collapses update history

This class is used in the implementation of `lamb.lang` binding operations. Here are several examples of the above calls with some further details on behavior. Note that the effect of these demo cells is dependent on run order!

In [91]:
a = lamb.meta.Assignment({'p': True, 'q': True})
a

[True/p,True/q]

In [92]:
a.modify(q=False)
a.modify(q=True)
a

[True/p,True/q][False/q][True/q]

In [93]:
a['q']

True

In [94]:
a.pop_child()
display(a, a['q'])

[True/p,True/q][False/q]

False

In [95]:
a.pop_child()
a

[True/p,True/q]

In [96]:
a2 = a.new_child(q=False, r=True)
a2

[True/p,True/q][False/q,True/r]

In [97]:
a2['q']

False

In [98]:
a2.flatten()

[True/p,False/q,True/r]

Modification overwrites without reference to what is overwritten:

In [99]:
a.modify(p='_c1') # no type constraints
display(a, a['p'])

[True/p,True/q][_c1/p]

_c1

On the other hand, the `merge` method works like `new_child`, but with constraints both on the type and structure of expressions. In a nutshell, this method lets (non-meta) terms merge with expressions of a compatible type, but doesn't let distinct complex expressions merge with each other. Merging specializes types if needed. More specifically, for each value in the assignment:

* If the old value is a non-meta term, and the new value is a term with a compatible type, the assignment gets the new value at the principal type.
* If the old value is a non-meta term, and the new value is either a meta-term or a non-term, then the assignment gets the new value.
* Otherwise, merging fails with a `TypeMismatch`.

Note that while the overall process is not symmetric, type resolution is symmetric. So for example a value of `x_e` can merge with a value of `x_X` resulting in a mapping to `x_e` (since `e` is the principal type).

Here are some examples:

In [101]:
a = lamb.meta.Assignment({'p': True, 'q': True})
with lamb.errors():
    a.merge(p='_c1') # type constraints enforced on update

<span style="color:red">**TypeMismatch**</span>: `True` and `_c1` conflict (Failed to merge typed expressions (incompatible types))

In [102]:
meta.Assignment(x=te('x_X')).merge(x=te('y_e')).merge(x=te('_c1'))

[x_X/x][y_e/x][_c1/x]

In [103]:
meta.Assignment(x=te('x_e')).merge(x=te('x_X')).merge(x=te('_c1'))

[x_e/x][x_e/x][_c1/x]

In [104]:
with lamb.errors():
    meta.Assignment(x=te('x_X')).merge(x=te('_c1')).merge(x=te('y_e'))

<span style="color:red">**TypeMismatch**</span>: `_c1` and `y_e` conflict (Failed to merge typed expressions; result is not equal)

### Truth tables and boolean evaluation

A pure boolean formula is one that consists only of boolean atoms (terms of type `t`) and boolean operators. A formula like this can be fully evaluated by providing an assignment that maps all terms in the formula to `MetaTerms` for type `t` (aka `True` and `False`). Let's call such an assignment *complete* relative to the formula. Simplification guarantees that an expression under a complete assignment is fully simplifiable to an atomic `MetaTerm`. Here is (another) example of evaluating a formula by providing a complete assignment:

In [105]:
te("~(p_t & q_t)").under_assignment({'p': True, 'q': False}).simplify_all()

True

The set of complete assignments for a formula, together with the evaluated results, amounts to what is usually called a *truth table* in the logic literature. The lambda notebook supports generating truth tables for boolean expressions via the function **`lamb.meta.truthtable`** (an alias to `lamb.meta.meta.truthtable`):

In [106]:
meta.truthtable(te("~(p_t & q_t)"))

| ${p}_{t}$ | ${q}_{t}$ | $\neg{} ({p}_{t} \wedge{} {q}_{t})$|
| :---:| :---:| :---:|
| 0| 0| 1|
| 0| 1| 1|
| 1| 0| 1|
| 1| 1| 0|


Reminder: metalanguage `0` and `1` are *not* equivalent to `False` and `True`, but they are used here because they make the resulting truth tables much more readable.

Here is a more complex example:

In [107]:
# a more complex case
meta.truthtable(te("(p_t => (q_t & r_t)) & ((q_t & r_t) => p_t)"))

| ${p}_{t}$ | ${q}_{t}$ | ${r}_{t}$ | $({p}_{t} \rightarrow{} ({q}_{t} \wedge{} {r}_{t})) \wedge{} (({q}_{t} \wedge{} {r}_{t}) \rightarrow{} {p}_{t})$|
| :---:| :---:| :---:| :---:|
| 0| 0| 0| 1|
| 0| 0| 1| 1|
| 0| 1| 0| 1|
| 0| 1| 1| 0|
| 1| 0| 0| 0|
| 1| 0| 1| 0|
| 1| 1| 0| 0|
| 1| 1| 1| 1|


Note that generating a truth table involves $2^n$ combinations of truth values for an expression with $n$ atomic terms; it therefore is very susceptible to combinatoric explosion. Most code that deals with this kind of exhaustive evaluation will by default cap the number of atomic terms allowed in order to avoid a stuck kernel with large memory use; supply named parameter `max_terms` to `truthtable` to override the default for this function (currently 12). Generally, working with small formulas in this way is completely tractable.

The function `lamb.meta.meta.truthtable_equiv` can be used to check if two expressions have equivalent truth tables; `lamb.meta.meta.truthtable_valid` to check if a truth table's values are entirely `True`, and `lamb.meta.meta.truthtable_contradictory` for the entirely `False` case. These are of course just as subject to combinatorial explosion as generating the underlying truth tables used for these checks. For example, here is an instance of De Morgan's verified by truth table:

In [108]:
meta.meta.truthtable_equiv(te("~(p_t & q_t)"), te("~p_t | ~q_t"))

True

**Complex and non-boolean subexpressions in truth tables**.
Unlike the standard logic textbook version of a truth table, lambda notebook truthtables support certain ways of handling non-pure boolean expressions. The first is via partial evaluation; if a subexpression can't be fully simplified by evaluating atomic boolean values, the subexpression will appear in the value column of a truth table. For example, in the following formula, when `p1` is `False`, the result can be completely determined, but if it is `True`, we would need to resolve `P(x)` to know the truth value.

In [109]:
meta.truthtable(te("p1_t & P_<e,t>(x)"))

| ${p1}_{t}$ | ${p1}_{t} \wedge{} {P}({x}_{e})$|
| :---:| :---:|
| 0| 0|
| 1| ${P}({x}_{e})$|


As another example, in the following expressions, there are two atomic propositions and two predicate-argument combinations; the latter can't be valued by truth table code. The result has a fully simplified `t` value in one row, and varying degrees of simplified formulas in the others.

In [110]:
meta.truthtable(te("p1_t & P_<e,t>(x) | p2_t & ~Q_<e,t>(x)"))

| ${p1}_{t}$ | ${p2}_{t}$ | $({p1}_{t} \wedge{} {P}({x}_{e})) \vee{} ({p2}_{t} \wedge{} \neg{} {Q}({x}_{e}))$|
| :---:| :---:| :---:|
| 0| 0| 0|
| 0| 1| $\neg{} {Q}({x}_{e})$|
| 1| 0| ${P}({x}_{e})$|
| 1| 1| ${P}({x}_{e}) \vee{} \neg{} {Q}({x}_{e})$|


The second means of handling complex subexpressions is via boolean "extraction". Given an arbitrary metalanguage expression of type `t`, *boolean extraction* produces a pure boolean formula and an assignment that will reconstruct the original formula. In a truth table context, this is used to treat all complex elements of type `t` like atoms, without worrying about how they are interpreted. The extracted form of the above example then has four evaluation columns instead of two, where the unanalyzed `P(x)` and `Q(x)` are treated like atoms:

In [111]:
meta.truthtable(te("p1_t & P_<e,t>(x) | p2_t & ~Q_<e,t>(x)"), extract=True)

| ${p1}_{t}$ | ${p2}_{t}$ | ${P}({x}_{e})$ | ${Q}({x}_{e})$ | $({p1}_{t} \wedge{} {P}({x}_{e})) \vee{} ({p2}_{t} \wedge{} \neg{} {Q}({x}_{e}))$|
| :---:| :---:| :---:| :---:| :---:|
| 0| 0| 0| 0| 0|
| 0| 0| 0| 1| 0|
| 0| 0| 1| 0| 0|
| 0| 0| 1| 1| 0|
| 0| 1| 0| 0| 1|
| 0| 1| 0| 1| 0|
| 0| 1| 1| 0| 1|
| 0| 1| 1| 1| 0|
| 1| 0| 0| 0| 0|
| 1| 0| 0| 1| 0|
| 1| 0| 1| 0| 1|
| 1| 0| 1| 1| 1|
| 1| 1| 0| 0| 1|
| 1| 1| 0| 1| 0|
| 1| 1| 1| 0| 1|
| 1| 1| 1| 1| 1|


Boolean extraction can be directly achieved via **`lamb.meta.meta.extract_boolean`**. This function takes a sequence of expressions, and returns a sequence of extracted expressions together with a single assignment that will reconstruct any of them. Subexpressions that are shared across the arguments will not be repeated, and the extracted forms will have the same names for these subexpressions. Any new term names are guaranteed to be unused in the input expressions.

In [112]:
e, m = meta.meta.extract_boolean(te("p1_t & P_<e,t>(x) | p2_t & ~Q_<e,t>(x)"))
display(e[0])
display(meta.Assignment(m)) # use Assignment for nicer rendering

((p1_t ∧ p3_t) ∨ (p2_t ∧ ¬p4_t))

[P_<e,t>(x_e)/p3,Q_<e,t>(x_e)/p4]

In [113]:
e[0].under_assignment(m)

((p1_t ∧ P_<e,t>(x_e)) ∨ (p2_t ∧ ¬Q_<e,t>(x_e)))

In [114]:
e, m = meta.meta.extract_boolean(te("P_<e,t>(x) & Q_<e,t>(x) & p_t"), te("Q_<e,t>(x)"))
display(e[0], e[1], meta.Assignment(m))

((p1_t ∧ p2_t) ∧ p_t)

p2_t

[P_<e,t>(x_e)/p1,Q_<e,t>(x_e)/p2]

In [115]:
display(*[x.under_assignment(m) for x in e])

((P_<e,t>(x_e) ∧ Q_<e,t>(x_e)) ∧ p_t)

Q_<e,t>(x_e)

Non-boolean values supplied to this function will result in an error. The limiting case of extraction is a single complex boolean expression, which will result in a trivial extracted frame with a one-element mapping (and a trivial truth table).

In [116]:
display(meta.meta.extract_boolean(te("Forall x_e : P_<e,t>(x)")))
meta.truthtable(te("Forall x_e : P_<e,t>(x)"), extract=True)

([p1_t], {'p1': (Forall x_e: P_<e,t>(x_e))})

| $\forall{} x_{e} \: . \: {P}({x})$ | $\forall{} x_{e} \: . \: {P}({x})$|
| :---:| :---:|
| 0| 0|
| 1| 1|


**Advanced API notes**: the return value of `truthtable` is an object of type `lamb.meta.meta.Evaluations`, which is a general purpose container for tracking and displaying boolean formula evaluations. A raw list of assignments (as `dict`s) can be generated via the function `lamb.meta.meta.truthtable_valuations`:

In [117]:
meta.meta.truthtable_valuations(te("p_t & q_t"))

[{'p': False, 'q': False},
 {'p': False, 'q': True},
 {'p': True, 'q': False},
 {'p': True, 'q': True}]

For doing any sort of exhaustive/brute force evaluation, the function `lamb.meta.meta.combinations` may be useful for generating arbitrary exhaustive valuations for a set of terms.

### Models

A **`meta.Model`** (alias for `meta.meta.Model`) is an object that can be used to evaluate arbitrary metalanguage expressions, patterned after the logical notion of a model. It is essentially an assignment paired with some domains for types. It is explicitly multi-sorted and does not require domains for all (or any) types. Here is a small example that defines a few entity constants and one property, over a domain of three entities:

In [165]:
reload_lamb()

In [166]:
m = meta.Model({'A': '_c1',
                'B': '_c2',
                'C': '_c1',
                'P': {'A', 'B'}},
               domain={'_c1', '_c2', '_c3'})
m

**Domain**: $D_{e} = \{\textsf{c3}_{e}, \textsf{c2}_{e}, \textsf{c1}_{e}\}$<br />**Valuations**:<br />$\left[\begin{array}{lll} A & \rightarrow & \textsf{c1}_{e} \\
B & \rightarrow & \textsf{c2}_{e} \\
C & \rightarrow & \textsf{c1}_{e} \\
P & \rightarrow & \textsf{Fun[\{B,A\}]}_{\left\langle{}e,t\right\rangle{}} \\ \end{array}\right]$

The constructor for `Model` takes an assignment (a `dict`/mapping) together with zero or more domain restrictions.

**Specifying assignments**. The assignment maps term names into domain elements or explicit `MetaTerm`s (but not other `TypedExpr`s). Constants may be used as shortcuts in subsequent definitions (going by order in the mapping), as exemplified above.

* Any domain elements may be used directly (see the documentation on Domain Elements and MetaTerms); in the above example `_c1` etc are domain elements for type `e`. This includes higher order domain elements such as functions.
* By default, functions may be defined for model values in the assignment by providing either a mapping (usually a `dict`), or in the case of a function mapping to `t` the set that the function would characterize.
* If you wish to use sets in your model, you can either construct them explicitly via a `MetaTerm` (which allows you to specify the exact type), or pass the named parameter `setfun=False` to the constructor, in which case a set in the assignment will be interpreted as a set.
* `MetaTerm`s are always allowed here, and this can lead to clearer but more verbose model definitions in terms of types. However, this syntax does not allow the use of constant names as shortcuts. For example, here is an equivalent statement of the above model.


In [167]:
meta.Model({'A': meta.MetaTerm('_c1', typ=tp("e")),
            'B': meta.MetaTerm('_c2', typ=tp("e")),
            'C': meta.MetaTerm('_c1', typ=tp("e")),
            'P': meta.MetaTerm({'_c1', '_c2'}, typ=tp("<e,t>"))},
            domain={'_c1', '_c2', '_c3'})

**Domain**: $D_{e} = \{\textsf{c3}_{e}, \textsf{c2}_{e}, \textsf{c1}_{e}\}$<br />**Valuations**:<br />$\left[\begin{array}{lll} A & \rightarrow & \textsf{c1}_{e} \\
B & \rightarrow & \textsf{c2}_{e} \\
C & \rightarrow & \textsf{c1}_{e} \\
P & \rightarrow & \textsf{Fun[\{B,A\}]}_{\left\langle{}e,t\right\rangle{}} \\ \end{array}\right]$

* Non-`MetaTerm` values are disallowed in `Model`, even if they are constants that could be immediately resolved. If you'd like to map to arbitrary `TypedExpr`s, simply use an `Assignment`. These can be used in conjunction with a `Model`; see Example 1 below.

In [168]:
with lamb.errors():
    m['x'] = te("f_<e,e>(A_e)")

with lamb.errors():
    m['x'] = te("A_e")

<span style="color:red">**ParseError**</span>: Unknown type domain element: `f_<e,e>(A_e), type e`

<span style="color:red">**ParseError**</span>: Unknown type domain element: `A_e`

**Specifying domains**. The `domain` parameter specifies zero or more domain restrictions for *atomic* type domains in the current type system. The value for `domain` can be `None` (meaning no restrictions), a single set, or a mapping of types to sets. If a single set is provided for the domain parameter, its type is inferred from the elements. To restrict multiple domains at once, use a dict(/mapping) from types to sets of elements of that type.

**Differences from standard logical models.** There are a number of specific ways in which this notion of model differs from standard logical presentations that it may be useful to be aware of.

* `Model` objects make no distinctions between constants and variables, and allow you to define either.
    - When a `Model` (or really, map constructed from one) is used as an assignment, constants can't be bound and won't shift, but this isn't enforced in any way by the model object itself. Via the `Model` object, constant values may be updated or changed through the usual interfaces.
* There is no constraint that a `Model` be complete relative to some (sub-)language, and a term being absent from the model results in that term not being evaluatable, nothing more.
* `Model` objects are embeddded in type domains, rather than defining them; they allow whatever sorts the underlying type system allows.
    - The absence of a model domain for some type indicates simply the absence of a domain restriction relative to the underlying type. (So for example, if type `e` is not restricted, it is treated as the default non-finite set of entities named `_c0`, `_c1`, ...)

**Advanced `Model` API notes.** 

* `Model`s are implemented with a `meta.meta.Assignment` as the underlying mapping object. `Model.assignment` accesses the underlying `Assignment` object for a model.
* The `strict_charfuns` (default: `True`) named parameter to the constructor determines the behavior of mapping-based characteristic functions relative to the model domain:
    - if it is `True`, they will be treated as partial and raise a `DomainError` if they are passed model domain elements that are not in their domain. (Non-characteristic functions always have this behavior.)
    - if it is `False`, they will return `False` for model domain elements not in their domain. This is convenient for using a mapping notation relative to very large (or notionally non-finite) type domains.
* `Model`s support the full `collections.abc.Mapping` API. Unlike `Assignment` (and `Namespace`) their values *must* be `MetaTerm`s. The shortcuts in construction can also be used in assignment, e.g. relative to the above model the string `A` will instantiate `MetaTerm(_c1)`.
* Trying to set a value to a non-meta `TypedExpr` will raise an exception. Models can be used as a base for a more flexible mapping by wrapping them in a [`ChainMap`](https://docs.python.org/3/library/collections.html#collections.ChainMap); when using a `Model` directly as an assignment (e.g. via `under_assignment`) this is taken care of automatically.

#### Evaluating expressions with a `Model`

One way to use a model is via its `evaluate` function. This function takes a typed expression, and uses the model's content to evaluate the expression by substituting terms and using the provided domain. By default this call triggers simplification after substitution; you can override this with the named parameter `simplify=False`. Model-defined constants evaluated this way are displayed in sans serif with their meta-value in a superscript.

In [169]:
m.evaluate(te("P_<e,t>(A_e)")).derivation

1. P_<e,t>(A_e)    (Assignment substitution)
 2. Fun[{_c2,_c1}](_c1)    (Reduction)
 3. True

As a shortcut, a `Model` object is callable, and calling it triggers the `evaluate` function.

In [170]:
m(te("P_<e,t>(A_e)")).derivation

1. P_<e,t>(A_e)    (Assignment substitution)
 2. Fun[{_c2,_c1}](_c1)    (Reduction)
 3. True

Because the domain is finite, quantified expressions can be evaluated on this model:

In [171]:
m.evaluate(te("Exists x_e : P_<e,t>(x)")).derivation.trace()

DisplayNode instance: HTML rendering only

In [124]:
m.evaluate(te("Forall x_e : P_<e,t>(x)")).derivation.trace()

DisplayNode instance: HTML rendering only

As noted above, there is no requirement that an expression's terms be present in the model, but of course in this case any missing terms can't be evaluated:

In [125]:
m.evaluate(te("Forall x_e : Q_<e,t>(x)"))

(Forall x_e: Q_<e,t>(x_e))

There is, on the other hand, a requirement on evaluation that any terms that have the same name as model-defined terms match in type. Mismatches will produce a `TypeMismatch` exception:

In [126]:
with lamb.errors():
    m.evaluate(te("P_<t,t>"))

<span style="color:red">**TypeMismatch**</span>: `P_<t,t>` and `Fun[{_c2,_c1}]` conflict (Variable replace failed with mismatched types)

Further notes on `evaluate`:

* This function can take an `Assignment` object, and will evaluate every value in the `Assignment`, returning a modified version.
* It is implemented using `under_assignment`, in contrast to the following technique.

#### Interpreting under a `Model` via context managers

The second way to use models is via a context manager using python's `with` syntax. (Like all context manager expressions, the final line of the indented block is not the cell's return value, so in these examples IPython's `display` is called directly.) In the scope of `with m.under()`, all of the assignments that make up the model are temporarily inserted into the metalanguage's global namespace, and domain restrictions are applied to the relevant type domains. Values in this namespace will directly impact any `te` magics or calls in the scope, and any `%lamb` line magics -- substitutions from the global namespace are used when building an expression (rather than substituted later).

In [127]:
with m.under():
    display(meta.global_namespace())
    x = %te simplify Forall x_e : P_<e,t>(x)
    display(x.derivation)

${A}_{e}\:=\:\textsf{c1}_{e}$<br />
${B}_{e}\:=\:\textsf{c2}_{e}$<br />
${C}_{e}\:=\:\textsf{c1}_{e}$<br />
${P}\:=\:\textsf{Fun[\{B,A\}]}_{\left\langle{}e,t\right\rangle{}}$

1. (Forall x_e: Fun[{B,A}](x_e))    (counterexample for ∀x)
 2. False

In order to make formulas easier to read, in the scope of a `m.under()` if the global namespace defines an entity constant, domain elements are rendered as that constant set in sans serif, with the actual value in a superscript. If there's more than one such definition, the first is used. 

In [128]:
with m.under():
    display(te("_c1"))

_c1

Python `with` expressions do not create a new scope, and variables defined in them persist past that scope. Any expressions created or evaluated (see below) in the scope of a model will therefore have the model-assigned values even if used outside the model's scope, though domain restrictions are purely temporary for regular expression creation and simplification. (Note: in contrast, compiled formulas are compiled with the current domain restriction!)

In [129]:
with m.under():
    x = %te P(A)
x.simplify_all().derivation

1. Fun[{_c2,_c1}](_c1)    (Reduction)
 2. True

A `Model`'s `under()` context manager is compatible with `as`, and when this syntax is used, it provides an object with two handy properties. First, the object allows (read-only) access to the model's valuations via attributes:

In [130]:
with m.under() as v:
    display(v.P)

Fun[{B,A}]

Second, the `as` object is callable, and calling it is a shortcut to calling `evaluate` from the model on a provided object. This can be quickly used to reevaluate a previously generated `TypedExpr` in the scope of a model.

*Related caveat*:  `TypedExpr` object created outside of the scope of a model or assignment will not automatically have its terms reinterpreted when used in the context of a model! For example, the `%te`-generated expression below has a type for `P` that is compatible with the model, but at the point of parsing it is just an arbitrary predicate. The `v(x)` call in the scope of a context manager evaluates (and simplifies) it:

In [131]:
x = %te P_<e,t>(_c1)
with m.under() as v:
    display(v(x).derivation)

1. P_<e,t>(_c1)    (Assignment substitution)
 2. Fun[{B,A}](_c1)    (Reduction)
 3. True

#### Modifying namespaces and type domains directly with a `Model`

A similar effect on assignments to the context manager approach can be achieved more permanently by modifying the namespace using a model. By default, the global namespace is an empty object of type `utils.Namespace`:

In [132]:
meta.global_namespace()

*(Empty Namespace)*

Calling `modify` on the namespace can be used to insert an assignment of any kind into that namespace, including a `Model`. (Running this cell repeatedly will make multiple modifications; the function does not check for duplicates.) Note that this does not make use of the model's domain!

In [133]:
meta.global_namespace().modify(m)
meta.global_namespace()

${A}_{e}\:=\:\textsf{c1}_{e}$<br />
${B}_{e}\:=\:\textsf{c2}_{e}$<br />
${C}_{e}\:=\:\textsf{c1}_{e}$<br />
${P}\:=\:\textsf{Fun[\{c2,c1\}]}_{\left\langle{}e,t\right\rangle{}}$

A very important thing to be aware of: modifications to the global namespace, while it is being modified by a `Model`, *will change that model*! To avoid this, modify with a copy using e.g. `m.copy()` instead of `m`.

In [134]:
meta.global_namespace()['C'] = '_c2'

In [135]:
m

**Domain**: $D_{e} = \{\textsf{c3}_{e}, \textsf{c2}_{e}, \textsf{c1}_{e}\}$<br />**Valuations**:<br />$\left[\begin{array}{lll} A & \rightarrow & \textsf{c1}_{e} \\
B & \rightarrow & \textsf{c2}_{e} \\
C & \rightarrow & \textsf{c2}_{e} \\
P & \rightarrow & \textsf{Fun[\{B,A\}]}_{\left\langle{}e,t\right\rangle{}} \\ \end{array}\right]$

A modification to a namespace can be removed by using `pop_child()`, or the namespace can be reset entirely with `reset()`. The current stack of modifications can be accessed  using the `maps` property.

In [136]:
meta.global_namespace().maps

[Model(assignment={'A': _c1, 'B': _c2, 'C': _c2, 'P': Fun[{_c2,_c1}]},
       domain={e: {_c1, _c2, _c3}},
       strict_charfuns=True),
 {}]

In [137]:
meta.global_namespace().reset() # or: meta.global_namespace().pop_child()
meta.global_namespace()

*(Empty Namespace)*

Similarly, to apply a model's type domain restrictions in a more lasting fashion, the function `modify_domains` on the type system can  be used. Similar to other `modify` calls, repeated calls to this function will stack even if they don't make a change. `reset_domains` restores every type to have completely unmodified domains.

In [138]:
meta.get_type_system().modify_domains(m)
tp("e").domain

Subdomain {'_c3', '_c2', '_c1'} of SimpleInfiniteSet('c')

In [139]:
meta.get_type_system().reset_domains()
tp("e").domain

SimpleInfiniteSet('c')

As a convenience, the function `apply` in a model makes both of these changes at once.

In [140]:
m.apply()

In [141]:
tp("e").domain

Subdomain {'_c3', '_c2', '_c1'} of SimpleInfiniteSet('c')

In [142]:
%te simplify Forall x_e : P_<e,t>(x)

False

In [144]:
# cleanup after m.apply() above
meta.get_type_system().reset_domains()
meta.global_namespace().reset()

#### Example from Chierchia and McConnell-Ginet

This example is from the Chierchia and McConnell-Ginet text *Meaning and Grammar: An Introduction to Semantics*, pp. 124ff.

In [145]:
reload_lamb()

First we instantiate the model. This is slightly different than the textbook version, in that we need to ground out constants in type domain elements, in this case `_c1`...`_c3`. Following the lambda notebook convention, we also use capitals for `j` and `m` to indicate that they are constants. (This is optional but does have cosmetic effects.)

In [146]:
m1 = meta.Model(
    {'Bond': '_c1',
     'Pavarotti': '_c2',
     'Loren': '_c3',
     'J': 'Bond',
     'M': 'Loren',
     'P': {'Loren', 'Pavarotti'},
     'Q': {'Loren', 'Bond'},
     'K': {('Bond', 'Bond'), ('Bond', 'Loren'), ('Loren', 'Pavarotti'), ('Pavarotti', 'Loren')},
     'G': {('Bond', 'Loren', 'Pavarotti'), ('Loren', 'Loren', 'Bond'),
           ('Loren', 'Bond', 'Pavarotti'), ('Pavarotti', 'Pavarotti', 'Loren')}
    },
    domain={'_c1', '_c2', '_c3'})
m1

**Domain**: $D_{e} = \{\textsf{c3}_{e}, \textsf{c2}_{e}, \textsf{c1}_{e}\}$<br />**Valuations**:<br />$\left[\begin{array}{lll} Bond & \rightarrow & \textsf{c1}_{e} \\
Pavarotti & \rightarrow & \textsf{c2}_{e} \\
Loren & \rightarrow & \textsf{c3}_{e} \\
J & \rightarrow & \textsf{c1}_{e} \\
M & \rightarrow & \textsf{c3}_{e} \\
P & \rightarrow & \textsf{Fun[\{Loren,Pavarotti\}]}_{\left\langle{}e,t\right\rangle{}} \\
Q & \rightarrow & \textsf{Fun[\{Loren,Bond\}]}_{\left\langle{}e,t\right\rangle{}} \\
K & \rightarrow & \textsf{Fun[\{(Bond,Loren),(Bond,Bond),(Loren,Pavarotti),(Pavarotti,Loren)\}]}_{\left\langle{}\left(e, e\right),t\right\rangle{}} \\
G & \rightarrow & \textsf{Fun[\{(Loren,Bond,Pavarotti),(Bond,Loren,Pavarotti),(Pavarotti,Pavarotti,Loren),(Loren,Loren,Bond)\}]}_{\left\langle{}\left(e, e, e\right),t\right\rangle{}} \\ \end{array}\right]$

The example involves combining a model with an assignment, something we can easily handle in the metalanguage. (Although in contrast to the more standard logical presentation in C&M-G, it is not actually necessary here.) The one caveat is that if the assignment is initially defined outside of the scope of a model, its terms will not automatically valued. This can be handled by passing the assignment directly to the model's evaluation function, which as you can see below, overrides the unvalued constants with appropriate model-derived values:

In [147]:
# with m1.under():
g1 = meta.Assignment(x1=te('Bond_e'), x2=te('Loren_e'), x3=te('Pavarotti_e')) # pure unvalued assignment
m1.evaluate(g1) # display the valued vesion of the assignment

[Bond_e/x1,Loren_e/x2,Pavarotti_e/x3][_c1/x1,_c3/x2,_c2/x3]

A simple formula can be interpreted on this model as above:

In [148]:
with m1.under():
    display(te("P(M)").simplify_all().derivation)

1. Fun[{Loren,Pavarotti}](_c3)    (Reduction)
 2. True

Now we go through several of the examples from the text and exercises. First, their example (20) (p.125). "$P(m) \wedge Q(x_3)$" translates to the metalanguage expression `P(M) & Q(x3)`, which we instiate with the `%te` magic in the scope of both the model and the assignment function, and then display its derivation. Note that the assignment here must be evaluated relative to the model, since it was defined absolutely.

In [164]:
# C&M-G example (20)
with m1.under() as m:
    with m(g1).under():
        e = %te simplify P(M) & Q(x3)
        display(e.derivation)

1. (Fun[{Loren,Pavarotti}](_c3) & Fun[{Loren,Bond}](_c2))    ([Reduction])
 2. (True & Fun[{Loren,Bond}](_c2))    ([Reduction])
 3. (True & False)    (alphabetic normalization)
 4. (False & True)    (conjunction (elimination on `False`))
 5. False

Next is exercise 3 problem (4). The formula is $G(x_1, j, x_1)$, which is straightforward to translate, but could be evaluated in multiple ways. If we follow the above paradigm, notice that terms and variables are substituted immediately with no derivation history (also seen above):

In [150]:
# ex. 3 (4)
with m1.under() as m:
    with m(g1).under():
        display(te("G(x1, J, x1)").reduce_all().derivation.trace())

DisplayNode instance: HTML rendering only

Alternatively, the assignment can be used more directly (again, it must itself be evaluated first):

In [151]:
# ex. 3 (4) version 2:
with m1.under() as m:
    display(te("G(x1, J, x1)").under_assignment(m(g1)).reduce_all().derivation.trace())

DisplayNode instance: HTML rendering only

Alternatively, both the model and assignment can be used directly. Here note that the formula is constructed in the abstract, and so types must be fully specified in order for evaluation/assignment to work.

In [152]:
e = %te G_<(e,e,e),t>(x1_e, J_e, x1_e)
m1.evaluate(e).under_assignment(m1.evaluate(g1)).reduce_all().derivation.trace()

DisplayNode instance: HTML rendering only

A somewhat more complex example: "$\neg P(x_1) \leftrightarrow K(x_2, j)$".

In [153]:
# ex. 3 (5)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify ~P(x1) <=> K(x2,J)
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only

The next two examples are even more complex. Note that there are some non-standard bits of notation in the original textbook formulas, as well as the use of an equality operator: $\neg [G(x_1, x_1, x1_) \rightarrow j=m] \wedge [\neg Q(j) \wedge K(j,m)]$

In [154]:
# ex. 3 (6)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify ~(G(x1,x1,x1) >> (J <=> M)) & (~Q(J) & K(J,M))
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only

$\neg x_1=j \rightarrow \neg G(x_1,x_1,x_1)$:

In [155]:
# ex. 3 (7)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify ~(x1 <=> J) >> ~G(x1,x1,x1)
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only

n.b. (8) has a typo (I think), so I have skipped it here.

Next we use this model for a few quantified expressions, starting with text example (22):

In [156]:
# (22)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify Exists x1 : P(x1)
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only

In [157]:
m1.evaluate(te("Exists x1 : P(x1)")).derivation.trace()

INFO (core): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x1_e'


DisplayNode instance: HTML rendering only

Finally, some multiply quantified examples. (In both of these cases, the truth value results from quantifying over the entire domain, so the derivation shows "generic on var" rather than printing a specific verifier/falsifier.)

In [158]:
# (23)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify Forall x1 : Exists x2 : K(x1,x2)
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only

In [159]:
# (24)
with m1.under() as m:
    with m(g1).under():
        x = %te simplify Exists x2 : Forall x1 : K(x1,x2)
        display(x.derivation.trace())

DisplayNode instance: HTML rendering only